In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/dataset/Quora Insincere Question/Quora-Insincere-Question.zip',compression='zip')

In [ ]:
data.head(5)

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np


In [ ]:
! pip install -U tensorflow-addons

Requirement already up-to-date: tensorflow-addons in /usr/local/lib/python3.7/dist-packages (0.12.1)


In [ ]:
import tensorflow_addons as tfa

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/dataset/Quora Insincere Question/Quora-Insincere-Question.zip',compression='zip')

In [ ]:
data.columns

Index(['qid', 'question_text', 'target'], dtype='object')

In [ ]:
data.drop(columns=['qid'],inplace=True)

In [ ]:
data['target'].value_counts()

0    1225312
1      80810
Name: target, dtype: int64

In [ ]:
data0=data[data['target']==0]
data1=data[data['target']==1]

In [ ]:
data00=data0[:30000]
data11=data1[:30000]

In [ ]:
data00[:10]

,question_text,target
0,How did Quebec nationalists see their province...,0
1,"Do you have an adopted dog, how would you enco...",0
2,Why does velocity affect time? Does velocity a...,0
3,How did Otto von Guericke used the Magdeburg h...,0
4,Can I convert montra helicon D to a mountain b...,0
5,"Is Gaza slowly becoming Auschwitz, Dachau or T...",0
6,Why does Quora automatically ban conservative ...,0
7,Is it crazy if I wash or wipe my groceries off...,0
8,"Is there such a thing as dressing moderately, ...",0
9,Is it just me or have you ever been in this ph...,0


In [ ]:
new_data=pd.concat([data00,data11])
new_data.reset_index(inplace=True)

In [ ]:
from sklearn.utils import shuffle
new_data = shuffle(new_data)

In [ ]:
new_data.reset_index(inplace=True)
new_data.drop(columns=['level_0','index'],inplace=True)

In [ ]:
new_data

,question_text,target
0,"How did you design a life for yourself, where ...",0
1,What approaches did J.R.R. Tolkien use to stud...,0
2,What is the estimated time to hike a 350 meter...,0
3,Did any black people contribute at all to huma...,1
4,Why do liberals always take the bait?,1
...,...,...
59995,How can mankind be so advanced but at the same...,1
59996,Why do evangelicals want to speed up the day o...,1
59997,Will N. Korea play Trump in the summit?,1
59998,Why is my son's stupid bangal cat always peein...,1


In [ ]:
import re
def preprocess(w):
  w=w.lower()
  w=re.sub(r"([?.!,%&^%$#@])",r" ",w)
  w=re.sub(r'[" "]+'," ",w)
  w=re.sub(r"[^a-zA-Z?.!,]+"," ",w)
  w=w.strip()
  return w

In [ ]:
ddata=list(map(preprocess,new_data['question_text']))

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices(ddata).batch(128)
vectorizer.adapt(text_ds)

In [ ]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset/GLOVE/glove.6B.100d.txt', sep=" ", quoting=3, header=None, index_col=0)
glove = {key: val.values for key, val in df.T.items()}

In [ ]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = glove.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 19271 words (729 misses)


In [ ]:
from tensorflow.keras.layers import Embedding
embedding_layer = Embedding(num_tokens,embedding_dim,embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),trainable=False,)

In [ ]:
from tensorflow.keras import layers
int_sequences_input = tf.keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = tf.keras.layers.LSTM(256, input_shape=(None,),return_sequences=True,recurrent_dropout=0.3,recurrent_regularizer=tf.keras.regularizers.L2(l2=0.01))(embedded_sequences)
#x = tf.keras.layers.LSTM(128,return_sequences=True,recurrent_dropout=0.3,recurrent_regularizer=tf.keras.regularizers.L2(l2=0.01))(x)
#x = tf.keras.layers.LSTM(128, return_sequences=True,recurrent_dropout=0.3,recurrent_regularizer=tf.keras.regularizers.L2(l2=0.01))(x)
x = tf.keras.layers.GlobalMaxPooling1D()(x)
x = tf.keras.layers.Dense(128, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
preds = tf.keras.layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(int_sequences_input, preds)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 100)         2000200   
_________________________________________________________________
lstm_3 (LSTM)                (None, None, 256)         365568    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129 

In [ ]:
len(ddata)

60000

In [ ]:
train_data=ddata[:45000]
train_labels=new_data['target'][:45000]
test_data=ddata[45000:]
test_labels=new_data['target'][45000:]

In [ ]:
len(new_data['target'])

60000

In [ ]:
x_train = vectorizer(np.array([[s] for s in train_data])).numpy()
x_val = vectorizer(np.array([[s] for s in test_data])).numpy()

y_train = np.array(train_labels)
y_val = np.array(test_labels)

In [ ]:
train_data[:10]


['how did you design a life for yourself where did you start what did you have and how did you make proceed what opportunities did you take',
 'what approaches did j r r tolkien use to study languages so effectively',
 'what is the estimated time to hike a meter hiking trip for a year old',
 'did any black people contribute at all to human history',
 'why do liberals always take the bait',
 'why do girls overwrite about themselves on quora',
 'do you find that liberals have an amazing way to rationalize and justify spending other people s income through taxes and social welfare can logic get any more convoluted why must welfare go to the middle class and not just the poor',
 'are you one of the scumbags who decided that it was a good idea to partake in adjacent reddit articles and the music thing on hn today',
 'why do white people like to take offense in somebody else s stead',
 'does anyone out there really think that the nfl owner player relationship is akin to a slave master and sl

In [ ]:
train_labels[:10]

0    0
1    0
2    0
3    1
4    1
5    1
6    1
7    1
8    1
9    1
Name: target, dtype: int64

In [ ]:
train_data

In [ ]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(), optimizer="adam", metrics=["acc"]
)
model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_val, y_val))